<a href="https://colab.research.google.com/gist/btahir/af1c57771fc4318792689a11122e6388/medium-backup-script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backup Your Medium Articles

This script is a quick and dirty way to backup the contents of your medium stories. 

In [0]:
# import stuff
import requests
import bs4
import os
import shutil
from PIL import Image

In [0]:
#@title Make A Folder
FOLDER_NAME = 'medium' #@param {type:"string"}
os.mkdir(FOLDER_NAME)

In [0]:
# cd into that directory
os.chdir(FOLDER_NAME)

In [0]:
#@title Enter Medium Story URL
MEDIUM_URL = 'https://medium.com/@btahir/using-python-to-backup-your-medium-articles-6f581a0c0d90' #@param {type:"string"}

response = requests.get(MEDIUM_URL)
soup = bs4.BeautifulSoup(response.text,'lxml')

In [5]:
# get title
title = soup.find('h1').get_text()
print(title)

Using Python to Backup your Medium Articles


In [0]:
# get text
paragraphs = soup.find_all('p')

with open('content.txt', 'w') as f:
  f.write(title + '\n\n')
  for p in paragraphs:
    f.write( p.get_text() )
    f.write('\n\n')

In [7]:
# get images
images = soup.find('body').find_all('img')
i = 0

for im in images:
  i+=1
  url = im.get('src')
  print('Downloading: ', url) 
  try:
    response = requests.get(url, stream=True)
    with open(str(i) + '.jpg', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
  except:
    print('Could not download: ', url)

Downloading:  https://miro.medium.com/fit/c/96/96/1*r5v9ysUAacoGCpxgMkjLiQ.jpeg
Downloading:  https://miro.medium.com/max/60/1*2qHXJ3-C6-RGphhlak9X5Q.jpeg?q=20
Downloading:  None
Could not download:  None
Downloading:  https://miro.medium.com/max/1400/1*2qHXJ3-C6-RGphhlak9X5Q.jpeg
Downloading:  https://miro.medium.com/fit/c/160/160/1*r5v9ysUAacoGCpxgMkjLiQ.jpeg


In [0]:
# delete smaller thumbnail photos - let's say any images below 500 height + width
for file in os.listdir('.'):
  if file.endswith(".jpg"):
    im = Image.open(file)
    size = im.height + im.width
    if size < 500:
      os.remove(file)

In [0]:
videos = soup.find_all('iframe')

youtubeStarterURL = 'https://www.youtube.com/watch?v='
vlist = []

for v in videos:
  try:
    src = v.get('src')
    start = src.index('watch%3Fv%3D')
    end = src.index('&image')
    vlist.append(youtubeStarterURL + src[start+12:end])
  except:
    continue

if len(vlist) > 0:
  with open('video-links.txt', 'w') as f:
    for vid in vlist:
      f.write( str(vid) )
      f.write('\n\n')

In [10]:
cd ..

/content


In [11]:
#@title ZIP Folder
ZIP_FILE = 'medium-zipped' #@param {type:"string"}
shutil.make_archive(ZIP_FILE, 'zip', FOLDER_NAME)

'/content/medium-zipped.zip'

You can download the final zipped file manually form the left or send to dropbox/google drive/ other storage place.